In [4]:
from fpdf import FPDF
from datetime import datetime
import unicodedata
import pandas as pd

VICINITY_ORANGE = (255, 102, 0)

def clean_text(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')

def load_commentary(filepath="VCX_commentary.txt"):
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            return clean_text(f.read())
    except FileNotFoundError:
        return "[Commentary file not found.]"

def load_dcf_summary(filepath="vcx_dcf.xlsx"):
    df = pd.read_excel(filepath, skiprows=150, nrows=8, usecols="D:F", header=None)
    df.columns = ["Metric", "Perpetuity", "EBITDA Exit"]
    return df

class AnalystReportPDF(FPDF):
    def header(self):
        self.set_font("Arial", 'B', 14)
        self.set_text_color(*VICINITY_ORANGE)
        self.cell(0, 10, self.company_header, ln=0)
        self.set_font("Arial", '', 12)
        self.set_text_color(0, 0, 0)
        self.cell(0, 10, "FINM3422 Equity Research Report", ln=1, align='R')
        self.set_font("Arial", '', 10)
        self.cell(0, 10, self.industry_line, ln=0)
        self.cell(0, 10, f"Report Date: {datetime.today().strftime('%Y-%b-%d')}", ln=1, align='R')
        self.set_draw_color(*VICINITY_ORANGE)
        self.set_line_width(1.5)
        self.line(10, self.get_y(), 200, self.get_y())
        self.ln(10)

    def footer(self):
        self.set_y(-15)
        self.set_font("Arial", "I", 8)
        self.set_text_color(100, 100, 100)
        self.cell(0, 10, f"Page {self.page_no()}", 0, 0, "C")

    def add_title(self, title):
        self.set_font("Arial", 'B', 16)
        self.set_text_color(*VICINITY_ORANGE)
        self.cell(0, 10, title, ln=True, align="L")
        self.ln(5)

    def add_paragraph(self, text):
        self.set_font("Arial", "", 12)
        self.set_text_color(0, 0, 0)
        self.multi_cell(0, 10, text)
        self.ln(2)

    def add_table(self, df):
        self.set_font("Arial", 'B', 12)
        self.set_fill_color(*VICINITY_ORANGE)
        self.set_text_color(255, 255, 255)
        col_width = self.w / 3.2
        for col in df.columns:
            self.cell(col_width, 10, str(col), border=1, align='C', fill=True)
        self.ln()
        self.set_font("Arial", '', 11)
        self.set_text_color(0, 0, 0)
        for _, row in df.iterrows():
            self.cell(col_width, 10, str(row["Metric"]), border=1)
            self.cell(col_width, 10, str(row["Perpetuity"]), border=1)
            self.cell(col_width, 10, str(row["EBITDA Exit"]), border=1)
            self.ln()
        self.ln(5)

def create_pdf(title, content, dcf_df, output_file="analyst_report.pdf"):
    pdf = AnalystReportPDF()
    pdf.company_header = "VICINITY CENTRES (VCX-AU)"
    pdf.industry_line = "Real Estate / Residential & Comm. REITs | GICS: Retail REITs"
    pdf.add_page()
    pdf.add_title(title)
    pdf.add_paragraph(content)
    pdf.add_title("DCF Summary Table")
    pdf.add_table(dcf_df)
    pdf.output(output_file)

commentary = load_commentary("VCX_commentary.txt")
dcf_data = load_dcf_summary("vcx_dcf.xlsx")
create_pdf("VCX: Analyst Commentary", commentary, dcf_data)
